In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf

from typing import Union, List, Tuple
from numpy.typing import ArrayLike, NDArray
from loguru import logger
from scipy.linalg import qr
from scipy.fft import irfft, rfftfreq
from IPython.display import Image, Audio
from pathlib import Path

from pyFDN.fdn import FDN
from pyFDN.feedback_matrix import FeedbackMatrix, FeedbackMatrixType
from pyFDN.delay_line import generate_coprime_delay_line_lengths
from pyFDN.utils import ms_to_samps, estimate_echo_density, conv_binaural, interaural_coherence_menzer
from pyFDN.sofa_parser import HRIRSOFAReader

#### The structure of the generalised Feedback Delay Network is shown below. In this notebook, we will play with the parameters of the FDN, and listen to its output. We will also binauralise it using interaural coherence matching.

In [ ]:
Image("../../resources/images/FDN_architecture.png")

\begin{aligned}
\boldsymbol{s}(n) & =\boldsymbol{A} \boldsymbol{s}(n-m)+\boldsymbol{b} x(n) \\
y(n) & =\boldsymbol{c}^T \boldsymbol{s}(n)+d x(n), \\
\boldsymbol{s}(n) & =\left[s_1(n), s_2(n), \ldots, s_N(n)\right]^T \\
\boldsymbol{s}(n-m) & =\left[s_1\left(n-m_1\right), s_2\left(n-m_2\right), \ldots, s_N\left(n-m_N\right)\right]^T
\end{aligned}

#### Define FDN parameters

In [ ]:
# sampling rate
fs = 48000
# number of delay lines
N = 8
frame_size = 2**8

# we want a binaural output
num_input = 1
num_output = 2

# input gains
b = np.random.randn(N, num_input)
# change coefficients of c so that the columns are orthonormal
C_ortho = FeedbackMatrix.get_random_unitary_matrix(N)
c = C_ortho[:, :num_output].T 
direct_gain = 0.5 * np.ones((num_output, num_input))

# delay lengths should be co-prime
# constrict delay range to be between 50 and 100ms
delay_range_ms = np.array([50, 100])
delay_lengths = generate_coprime_delay_line_lengths(delay_range_ms, N, fs)
logger.info(f'The delay line lengths are {delay_lengths} samples')

# how long should the impulse response be
ir_len = ms_to_samps(300, fs)
# create an impulse
input_data = np.zeros((num_input, ir_len))
input_data[:, 0] = 1.0

# desired broadband T60
des_t60_ms = 500

#### Plot FDN IR for a scalar feedback matrix

In [ ]:
fdn = FDN(fs, num_input, num_output, N, frame_size)
fdn.init_io_gains(b, c)
fdn.init_direct_gain(direct_gain)
fdn.init_delay_line_lengths(delay_lengths)
fdn.init_feedback_matrix()
fdn.init_absorption_gains(des_t60_ms)
fdn.init_delay_lines()

sfm_fdn_ir = fdn.process(input_data)
del fdn

In [ ]:
time_vector = np.arange(0, ir_len/fs, 1.0/fs)
time_constant = (des_t60_ms*1e-3) / np.log(1000) 
exp_envelope = np.exp(-time_vector / time_constant)

plt.figure()
plt.plot(time_vector, sfm_fdn_ir.T)
plt.plot(time_vector, np.stack((exp_envelope, -exp_envelope), axis=-1), 'k--')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'FDN IR with scalar feedback matrix and T60 of {des_t60_ms} ms')

#### Plot FDN IR for a filter feedback matrix

In [ ]:
fdn = FDN(fs, num_input, num_output, N, frame_size)
fdn.init_io_gains(b, c)
fdn.init_direct_gain(direct_gain)
fdn.init_delay_line_lengths(delay_lengths)
fdn.init_feedback_matrix(FeedbackMatrixType.FILTER_VELVET, sparsity=0.2, num_mixing_stages=1)
fdn.init_absorption_gains(des_t60_ms)
fdn.init_delay_lines()

vfm_fdn_ir = fdn.process(input_data)

In [ ]:
fig, ax = plt.subplots(N, N, figsize=(12,12))
for i in range(N):
    for j in range(N):
        ax[i,j].stem(fdn.feedback.feedback_matrix[i, j, :])
        if j < N:
            ax[i,j].set_xlabel('')
        if i > 0:
            ax[i,j].set_ylabel('')
fig.suptitle("Velvet feedback matrix")
fig.tight_layout()

plt.figure()
plt.plot(time_vector, vfm_fdn_ir.T)
plt.plot(time_vector, np.stack((exp_envelope, -exp_envelope), axis=-1), 'k--')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'FDN IR with velvet feedback matrix and T60 of {des_t60_ms} ms')

del fdn

#### Compare the echo densities of the IRs with different feedback matrices

In [ ]:
ned_sfm = estimate_echo_density(sfm_fdn_ir.T, fs)
ned_vfm = estimate_echo_density(vfm_fdn_ir.T, fs)

plt.figure()
plt.plot(time_vector, ned_sfm, 'b')
plt.plot(time_vector, ned_vfm, 'r')
plt.xlabel('Time (s)')
plt.ylabel('NED')

#### Listen to the 2 IRs convolved with a dry signal

In [ ]:
audio_path = Path('../../resources/audio/')
input_signal, fs = sf.read(str(audio_path / 'lexicon_dry15.wav'))
Audio(str(audio_path / 'lexicon_dry15.wav'))

In [ ]:
output_sfm_fdn = conv_binaural(sfm_fdn_ir.T, input_signal, ear_axis=-1)
sf.write(str(audio_path / 'lexicon_sfm_fdn.wav'), output_sfm_fdn, fs)
Audio(str(audio_path / 'lexicon_sfm_fdn.wav'))

In [ ]:
output_vfm_fdn = conv_binaural(vfm_fdn_ir.T, input_signal, ear_axis=-1)
sf.write(str(audio_path / 'lexicon_vfm_fdn.wav'), output_vfm_fdn, fs)
Audio(str(audio_path / 'lexicon_vfm_fdn.wav'))

### Binauralise the FDN output by applying the correct IAC filter to the two output channels

To get the interaural coherence we desire, we will read an HRTF dataset of a KEMAR mannequin and calculate the IAC from the left and right ear signals, $L(\omega)$ and $R(\omega)$ over all azimuth and elevation angles ($M$).

$$ \Phi(\omega) = \frac{|\sum_{m=1}^M L_m(\omega) R^*_m(\omega)|}{\sqrt{\sum_{m=1}^M |L_m(\omega)|^2 \sum_{m=1}^M |R_m(\omega)|^2}}
$$

We can then get two filters applied to both FDN output channels, given by:
$$ U(\omega) = \sqrt{\frac{1 + \Phi(\omega)}{2}} \qquad V(\omega) = \sqrt{\frac{1 - \Phi(\omega)}{2}}$$

We can take the inverse DFT to these to get $u(n)$ and $v(n)$ in the time domain. If the two FDN output channels are $r_1(n)$ and $r_2(n)$ respectively, the final output is given by:

\begin{align}
b_L(n) = (u * r_1 + v * r_2)(n) \\
b_R (n) = (u * r_1 - v*r_2) (n)
\end{align}
This operation can be written in matrix form as:

$$
\begin{bmatrix} b_L (n) \\ b_R(n)\end{bmatrix} = \begin{bmatrix} u(n) & v(n) \\ u(n) & -v(n)\end{bmatrix} * \begin{bmatrix}r_1(n) \\ r_2(n) \end{bmatrix}
$$
Now, the interaural coherence of the output is:

\begin{align}
\hat{\Phi}(\omega) &= \frac{\mathbb{E}\left(B_L(\omega) B_R^*(\omega) \right)}{\mathbb{E}\left( |B_L(\omega)|\right) \mathbb{E}\left( |B_R(\omega)|\right)} \\
&= \frac{\mathbb{E} \left[(U(\omega)R_1(\omega) + V(\omega)R_2(\omega))(U^*(\omega) R_1^*(\omega) - V^*(\omega) R_2^*(\omega)) \right]}{\mathbb{E} \left( |U(\omega)|^2 |R_1(\omega)^2| + |V(\omega)|^2 |R_2(\omega)|^2\right)} \\
&= \frac{|U(\omega)|^2 |\mathbb{E}\left(|R_1(\omega)|^2 \right) - |V(\omega)|^2 \mathbb{E}\left( |R_2(\omega)|^2 \right)}{ |U(\omega)|^2 |\mathbb{E}\left(|R_1(\omega)|^2 \right) + |V(\omega)|^2 \mathbb{E}\left(|R_2(\omega)|^2 \right)}
\end{align}

Now, the two outputs from the FDN are uncorrelated and have the same power, i.e, $\mathbb{E}\left[R_1 (\omega) R_2^*(\omega) \right] = 0$ and $\mathbb{E}\left[|R_1(\omega)|^2 \right] = \mathbb{E}\left[ |R_2(\omega)|^2 \right] = P(\omega)$. Therefore, the achieved IAC is:

\begin{align}
\hat{\Phi}(\omega) &= \frac{\left[\frac{1 + \Phi(\omega)}{2} - \frac{1 - \Phi(\omega)}{2} \right]P(\omega)}{\left[ \frac{1 + \Phi(\omega)}{2} + \frac{1 - \Phi(\omega)}{2} \right] P(\omega)}\\
\hat{\Phi}(\omega) &= \Phi(\omega)
\end{align}

#### Read SONICOM KEMAR small ears SOFA file and get the desired IAC shape

In [ ]:
sofa_path = '../../resources/KEMAR/KEMAR_Knowl_EarSim_SmallEars/HRTF/HRTF/48kHz/KEMAR_Knowl_EarSim_SmallEars_FreeFieldComp_48kHz.sofa'
sofa_file = HRIRSOFAReader(sofa_path)
# caculate IAC from the SOFA file
des_iac = sofa_file.get_interaural_coherence()
num_fft_bins = sofa_file.ir_length

#### Create the FDN postprocessing filter matrix

In [ ]:
u = irfft(np.sqrt((1 + des_iac) / 2), n = num_fft_bins)
v = irfft(np.sqrt((1 - des_iac) / 2), n = num_fft_bins)

# create a matrix with these filters
iac_filt_matrix = np.zeros((num_output, num_output, num_fft_bins), dtype=float)
iac_filt_matrix[0, ...] = np.vstack((u, u))
iac_filt_matrix[1, ...] = np.vstack((v, -v))

#### Run the FDN with postprocessing filters

In [ ]:
fdn = FDN(fs, num_input, num_output, N, frame_size)

fdn.init_io_gains(b, c)
fdn.init_direct_gain(direct_gain)
fdn.init_delay_line_lengths(delay_lengths)
fdn.init_feedback_matrix(FeedbackMatrixType.FILTER_VELVET, sparsity=0.2, num_mixing_stages=1)
fdn.init_absorption_gains(des_t60_ms)
fdn.init_delay_lines()
fdn.init_postprocessing_filters(iac_filt_matrix)

bin_vfm_fdn_ir = fdn.process(input_data)
output_bin_vfm_fdn = conv_binaural(bin_vfm_fdn_ir.T, input_signal, ear_axis=-1)
sf.write(str(audio_path / 'lexicon_bin_vfm_fdn.wav'), output_bin_vfm_fdn, fs)
Audio(str(audio_path / 'lexicon_bin_vfm_fdn.wav'))

plt.figure()
plt.plot(bin_vfm_fdn_ir.T)
plt.plot(np.stack((exp_envelope, -exp_envelope), axis=-1), 'k--')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

#### Plot the desired and achieved IACs - low-frequency IAC is off but high-frequency IAC matches better.
- FDN output channels are not fully decorrelated to begin with so the math falls apart.

In [ ]:
from importlib import reload
import pyFDN
reload(pyFDN.utils)
from pyFDN.utils import interaural_coherence_menzer

des_iac_freqs = rfftfreq(num_fft_bins, d = 1.0 / fs)
init_iac, est_iac_freqs = interaural_coherence_menzer(vfm_fdn_ir, fs, time_axis=-1, ear_axis=0)
est_iac, _ = interaural_coherence_menzer(bin_vfm_fdn_ir, fs, time_axis=-1, ear_axis=0)

plt.figure()
plt.semilogx(des_iac_freqs, des_iac)
plt.semilogx(est_iac_freqs, init_iac)
plt.semilogx(est_iac_freqs, est_iac)
plt.xlabel('Frequency (Hz)')
plt.xlim([20, 20000])
plt.ylabel('IAC')
plt.legend(['Desired', 'Initial', 'Achieved'])